In [ ]:
sparql = """
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix sr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
prefix ukhpi: <http://landregistry.data.gov.uk/def/ukhpi/>
prefix lrppi: <http://landregistry.data.gov.uk/def/ppi/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix lrcommon: <http://landregistry.data.gov.uk/def/common/>

# Returns the Price Paid data from the default graph for each transaction record having
# an address with the given postcode.
# The postcode to query is set using SPARQL 1.1's 'values' clause

SELECT ?paon ?saon ?street ?town ?county ?postcode ?amount ?date ?category
WHERE
{
  VALUES ?postcode {"SE15 2RS"^^xsd:string}
  
  filter contains(?postcode,"SE15")  

  ?addr lrcommon:postcode ?postcode.

  ?transx lrppi:propertyAddress ?addr ;
          lrppi:pricePaid ?amount ;
          lrppi:transactionDate ?date ;
          lrppi:transactionCategory/skos:prefLabel ?category.

  OPTIONAL {?addr lrcommon:county ?county}
  OPTIONAL {?addr lrcommon:paon ?paon}
  OPTIONAL {?addr lrcommon:saon ?saon}
  OPTIONAL {?addr lrcommon:street ?street}
  OPTIONAL {?addr lrcommon:town ?town}
}
ORDER BY ?amount
"""

In [ ]:
url = "http://landregistry.data.gov.uk/landregistry/query"

In [ ]:
import requests

In [ ]:
payload = {"query": sparql}
res = requests.post(url, payload)

In [ ]:
import json
results = json.loads(res.text)

In [ ]:
bindings = results['results']['bindings']

for_pd = []
for result in bindings:
    new_obj = {}
    for key in result:
        new_obj[key] = result[key]['value']
    for_pd.append(new_obj)

In [ ]:
import pandas as pd 
df = pd.DataFrame(for_pd)
df.head(2)

In [ ]:
import altair as alt

In [ ]:
alt.Chart(df).mark_bar().encode(x="year(date):T",y='count()')